In [240]:
import pandas as pd
import numpy as np


train_df = pd.read_csv("../data/processed_data/train.csv")
test_df = pd.read_csv("../data/original_data/ais_test.csv")
#schedules_df = pd.read_csv("../data/processed_data/schedules.csv")

#print(len(schedules_df))

#schedules_df.head()


In [241]:

reference_start_time = pd.to_datetime("2024-01-01 00:00:00")
reference_end_time = pd.to_datetime("2025-01-01 00:00:00")  # Might need to adjust the end time
total_time_span = (reference_end_time - reference_start_time).total_seconds()

In [242]:

#schedules_df['sailing_time_converted'] = pd.to_datetime(schedules_df['sailingDate'], errors='coerce')
#schedules_df['arrival_time_converted'] = pd.to_datetime(schedules_df['arrivalDate'], errors='coerce')
#schedules_df = schedules_df.sort_values(by=['sailing_time_converted']).reset_index(drop=True)

# Might have to make this timezone aware!!
#schedules_df['sailing_time_converted'] = schedules_df['sailing_time_converted'].dt.tz_localize(None)
#schedules_df['arrival_time_converted'] = schedules_df['arrival_time_converted'].dt.tz_localize(None)

#schedules_df = schedules_df[schedules_df["sailing_time_converted"] <= reference_end_time]
#schedules_df = schedules_df[schedules_df["sailing_time_converted"] >= reference_start_time]
#schedules_df = schedules_df[schedules_df["sailing_time_converted"] < schedules_df["arrival_time_converted"]]


#schedules_df.drop(['sailing_time_converted'], axis=1, inplace=True)
#schedules_df.drop(['arrival_time_converted'], axis=1, inplace=True)


#schedules_df.tail()

In [243]:
# Training data - time feature conversion and normalization
train_df['original_time_converted'] = pd.to_datetime(train_df['time'], errors='coerce')
train_df['time_position_1_step_ago_converted'] = pd.to_datetime(train_df['time_position_1_step_ago'], errors='coerce')
train_df['time_position_2_steps_ago_converted'] = pd.to_datetime(train_df['time_position_2_steps_ago'], errors='coerce')


train_df['time'] = (train_df['original_time_converted'] - reference_start_time).dt.total_seconds()
train_df['time_position_1_step_ago'] = (train_df['time_position_1_step_ago_converted'] - reference_start_time).dt.total_seconds()
train_df['time_position_2_steps_ago'] = (train_df['time_position_2_steps_ago_converted'] - reference_start_time).dt.total_seconds()


# Normalize time (between 0 and 1)
train_df['time'] = train_df['time'] / total_time_span
train_df['time_position_1_step_ago'] = train_df['time_position_1_step_ago'] / total_time_span
train_df['time_position_2_steps_ago'] = train_df['time_position_2_steps_ago'] / total_time_span

# Add new features
train_df['week_of_the_year'] = train_df['original_time_converted'].dt.isocalendar().week  # Week number of the year
train_df['day_of_the_year'] = train_df['original_time_converted'].dt.dayofyear

# Normalize other features
# Min-max normalization to scale features between 0 and 1
train_df['week_of_the_year'] = (train_df['week_of_the_year'] - 1) / 52  # Normalize week_of_the_year (1-53)
train_df['day_of_the_year'] = (train_df['day_of_the_year'] - 1) / 365  # Normalize day_of_the_year (1-365)

# Drop intermediate columns
train_df.drop(['original_time_converted', 'time_position_1_step_ago_converted', 'time_position_2_steps_ago_converted'], axis=1, inplace=True)

train_df.head()


,time,cog,sog,rot,heading,navstat,etaRaw,latitude,longitude,vesselId,portId,latitude_1_step_ago,longitude_1_step_ago,time_position_1_step_ago,latitude_2_steps_ago,longitude_2_steps_ago,time_position_2_steps_ago,week_of_the_year,day_of_the_year
0,0.031663,308.1,17.1,-6,316,0,01-08 06:00,7.50361,77.58340,61e9f38eb937134a3c4bfd8b,61d376b393c6feb83e5eb50c,NaN,NaN,NaN,NaN,NaN,NaN,0.019231,0.030137
1,0.031707,307.6,17.3,5,313,0,01-14 23:30,7.57302,77.49505,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,7.50361,77.58340,0.031663,NaN,NaN,NaN,0.019231,0.030137
2,0.031757,306.8,16.9,5,312,0,01-14 23:30,7.65043,77.39404,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,7.57302,77.49505,0.031707,7.50361,77.58340,0.031663,0.019231,0.030137
3,0.031798,307.9,16.9,6,313,0,01-14 23:30,7.71275,77.31394,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,7.65043,77.39404,0.031757,7.57302,77.49505,0.031707,0.019231,0.030137
4,0.031838,307.0,16.3,7,313,0,01-14 23:30,7.77191,77.23585,61e9f38eb937134a3c4bfd8b,61d376d893c6feb83e5eb546,7.71275,77.31394,0.031798,7.65043,77.39404,0.031757,0.019231,0.030137


In [244]:
# Test data - time feature conversion and normalization
test_df['time_converted'] = pd.to_datetime(test_df['time'], errors='coerce')
test_df['time'] = (test_df['time_converted'] - reference_start_time).dt.total_seconds()
test_df['time'] = test_df['time'] / total_time_span

# Add new features
test_df['week_of_the_year'] = test_df['time_converted'].dt.isocalendar().week  # Week number of the year
test_df['day_of_the_year'] = test_df['time_converted'].dt.dayofyear

# Normalize other features in test data
test_df['week_of_the_year'] = (test_df['week_of_the_year'] - 1) / 52  # Normalize week_of_the_year (1-53)
test_df['day_of_the_year'] = (test_df['day_of_the_year'] - 1) / 365  # Normalize day_of_the_year (1-365)

# Drop intermediate columns
test_df.drop(['time_converted'], axis=1, inplace=True)

test_df.head()

,ID,vesselId,time,scaling_factor,week_of_the_year,day_of_the_year
0,0,61e9f3aeb937134a3c4bfe3d,0.349733,0.3,0.346154,0.350685
1,1,61e9f473b937134a3c4c02df,0.349739,0.3,0.346154,0.350685
2,2,61e9f469b937134a3c4c029b,0.349746,0.3,0.346154,0.350685
3,3,61e9f45bb937134a3c4c0221,0.349747,0.3,0.346154,0.350685
4,4,61e9f38eb937134a3c4bfd8d,0.349750,0.3,0.346154,0.350685


In [245]:
# Schedule data - time feature conversion and normalization
#schedules_df['sailing_time_converted'] = pd.to_datetime(schedules_df['sailingDate'], errors='coerce').dt.tz_localize(None) # Might have to make this timezone aware!!
#schedules_df['arrival_time_converted'] = pd.to_datetime(schedules_df['arrivalDate'], errors='coerce').dt.tz_localize(None) # Might have to make this timezone aware!!
#schedules_df['sailingDate'] = (schedules_df['sailing_time_converted'] - reference_start_time).dt.total_seconds()
#schedules_df['arrivalDate'] = (schedules_df['arrival_time_converted'] - reference_start_time).dt.total_seconds()

#schedules_df['sailingDate'] = schedules_df['sailingDate'] / total_time_span
#schedules_df['arrivalDate'] = schedules_df['arrivalDate'] / total_time_span


# Add new features
#schedules_df['sailing_week_of_the_year'] = schedules_df['sailing_time_converted'].dt.isocalendar().week  # Week number of the year
#schedules_df['sailing_day_of_the_year'] = schedules_df['sailing_time_converted'].dt.dayofyear
#schedules_df['arrival_week_of_the_year'] = schedules_df['arrival_time_converted'].dt.isocalendar().week  # Week number of the year
#schedules_df['arrival_day_of_the_year'] = schedules_df['arrival_time_converted'].dt.dayofyear

# Normalize other features in test data
#schedules_df['sailing_week_of_the_year'] = (schedules_df['sailing_week_of_the_year'] - 1) / 52  # Normalize week_of_the_year (1-53)
#schedules_df['sailing_day_of_the_year'] = (schedules_df['sailing_day_of_the_year'] - 1) / 365  # Normalize day_of_the_year (1-365)
#schedules_df['arrival_week_of_the_year'] = (schedules_df['arrival_week_of_the_year'] - 1) / 52  # Normalize week_of_the_year (1-53)
#schedules_df['arrival_day_of_the_year'] = (schedules_df['arrival_day_of_the_year'] - 1) / 365  # Normalize day_of_the_year (1-365)

# Drop intermediate columns
#schedules_df.drop(['sailing_time_converted'], axis=1, inplace=True)
#schedules_df.drop(['arrival_time_converted'], axis=1, inplace=True)

#schedules_df.head()


In [246]:
train_df.to_csv('../data/processed_data/train.csv', index=False)
test_df.to_csv("../data/processed_data/test.csv", index=False)
#schedules_df.to_csv("../data/processed_data/schedules.csv", index=False)